In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

def string_to_binary(message):
    """Convert a string message to binary representation"""
    binary = ''.join(format(ord(char), '08b') for char in message)
    return binary

def binary_to_string(binary):
    """Convert binary back to string"""
    # Ensure the binary string length is a multiple of 8
    binary = binary + '0' * (8 - len(binary) % 8) if len(binary) % 8 != 0 else binary
    
    # Convert each 8-bit chunk back to a character
    message = ''
    for i in range(0, len(binary), 8):
        byte = binary[i:i+8]
        message += chr(int(byte, 2))
    
    # Remove null characters at the end (if any)
    return message.split('\x00')[0]

def hide_message(image, message, bits_used=2):
    """
    Hide a message in the least significant bits of the image
    
    Parameters:
    - image: The input image
    - message: The message to hide
    - bits_used: Number of LSBs to use for hiding (1 or 2)
    
    Returns:
    - Modified image with hidden message
    """
    # Make a copy of the image
    output = image.copy()
    
    # Convert message to binary
    binary_message = string_to_binary(message)
    
    # Add delimiter to know when message ends
    binary_message += '11111111'
    
    # Calculate maximum message size that can be embedded
    max_bytes = (image.shape[0] * image.shape[1] * 3 * bits_used) // 8
    if len(binary_message) > max_bytes * 8:
        raise ValueError(f"Message too long. Maximum {max_bytes} bytes allowed.")
    
    # Flatten the image
    output_flat = output.reshape(-1)
    
    # Define the bit mask based on number of bits used
    if bits_used == 1:
        mask = 0xFE  # 11111110 in binary
        divisor = 1
    else:  # bits_used == 2
        mask = 0xFC  # 11111100 in binary
        divisor = 2
    
    # Embed the message
    idx = 0
    for i in range(len(binary_message)):
        # Get the position in the flattened array
        pos = idx // divisor
        
        # If we've reached the end of the array, break
        if pos >= len(output_flat):
            break
        
        # Clear the last bits_used bits of the pixel value
        output_flat[pos] = output_flat[pos] & mask
        
        # Set the last bits_used bits to the message bits
        if bits_used == 1:
            output_flat[pos] = output_flat[pos] | int(binary_message[i])
        else:  # bits_used == 2
            if i % 2 == 0 and i+1 < len(binary_message):
                # Combine two bits from the message
                bits = int(binary_message[i:i+2], 2)
                output_flat[pos] = output_flat[pos] | bits
                i += 1  # Skip the next bit as we've already used it
        
        idx += bits_used
    
    # Reshape back to original image dimensions
    output = output_flat.reshape(image.shape)
    
    return output

def extract_message(image, bits_used=2):
    """
    Extract a hidden message from the least significant bits of the image
    
    Parameters:
    - image: Image with hidden message
    - bits_used: Number of LSBs used for hiding (1 or 2)
    
    Returns:
    - Extracted message
    """
    # Flatten the image
    flat_image = image.reshape(-1)
    
    # Create a bit mask based on bits_used
    if bits_used == 1:
        mask = 0x01  # 00000001 in binary
    else:  # bits_used == 2
        mask = 0x03  # 00000011 in binary
    
    # Extract the message bits
    binary = ""
    delimiter = "11111111"  # 8 ones to mark end of message
    
    i = 0
    while i < len(flat_image):
        # Extract bits
        if bits_used == 1:
            # Extract 1 bit
            bit = flat_image[i] & mask
            binary += str(bit)
        else:  # bits_used == 2
            # Extract 2 bits
            bits = flat_image[i] & mask
            binary += format(bits, f'0{bits_used}b')
        
        # Check for delimiter every 8 bits
        if len(binary) >= 8 and binary[-8:] == delimiter:
            binary = binary[:-8]  # Remove the delimiter
            break
        
        i += 1
    
    # Convert binary to string
    message = binary_to_string(binary)
    
    return message

def main():
    print("Steganography Application")
    print("------------------------")
    
    # Step 1: Load existing image
    print("\nStep 1: Loading existing selfie.jpg")
    image_path = 'selfie.jpg'
    
    if not os.path.exists(image_path):
        print(f"Error: {image_path} not found in the current directory.")
        return
    
    selfie = cv2.imread(image_path)
    if selfie is None:
        print(f"Error: Could not load {image_path}. Make sure it's a valid image file.")
        return
    
    original = selfie.copy()
    
    # Step 2: Get message from user
    print("\nStep 2: Enter message to hide")
    message = input("Enter the message to hide: ")
    
    # Step 3: Choose number of bits (1 or 2)
    bits_choice = input("Use 1 or 2 bits for hiding? (1/2) [default: 2]: ")
    bits_used = 1 if bits_choice == '1' else 2
    
    # Justify the choice
    print(f"\nUsing {bits_used} least significant bits for hiding the message.")
    print(f"Justification for using {bits_used} bits:")
    
    if bits_used == 1:
        print("- Better image quality (less visual distortion)")
        print("- Lower capacity (can hide less data)")
        print("- Higher security (harder to detect visually)")
    else:
        print("- Higher capacity (can hide more data)")
        print("- Slightly lower image quality")
        print("- Still provides good security with minimal visual distortion")
    
    max_bytes = (selfie.shape[0] * selfie.shape[1] * 3 * bits_used) // 8
    print(f"- Maximum message size with this image: {max_bytes} bytes")
    print(f"- Your message size: {len(message)} bytes")
    
    # Step 4: Hide the message
    print("\nStep 3: Hiding message in image...")
    try:
        output = hide_message(selfie, message, bits_used)
        print("Message hidden successfully!")
    except ValueError as e:
        print(f"Error: {e}")
        return
    
    # Step 5: Show both images
    print("\nStep 4: Displaying original and modified images...")
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.title("Image with Hidden Message")
    plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Step 6: Extract and display the hidden message
    print("\nStep 5: Extracting hidden message...")
    extracted = extract_message(output, bits_used)
    print(f"Extracted message: {extracted}")
    
    # Step 7: Save the modified image
    save_choice = input("\nWould you like to save the modified image? (y/n): ")
    if save_choice.lower() == 'y':
        cv2.imwrite("steganography_output.png", output)
        print("Modified image saved as 'steganography_output.png'")
    
    print("\nSteganography process completed.")

if __name__ == "__main__":
    main()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

def string_to_binary(message):
    """Convert a string message to binary representation"""
    binary = ''.join(format(ord(char), '08b') for char in message)
    return binary

def binary_to_string(binary):
    """Convert binary back to string"""
    # Ensure the binary string length is a multiple of 8
    binary = binary + '0' * (8 - len(binary) % 8) if len(binary) % 8 != 0 else binary
    
    # Convert each 8-bit chunk back to a character
    message = ''
    for i in range(0, len(binary), 8):
        byte = binary[i:i+8]
        message += chr(int(byte, 2))
    
    # Remove null characters at the end (if any)
    return message.split('\x00')[0]

def hide_message(image, message, bits_used=2):
    """
    Hide a message in the least significant bits of the image
    
    Parameters:
    - image: The input image
    - message: The message to hide
    - bits_used: Number of LSBs to use for hiding (1 or 2)
    
    Returns:
    - Modified image with hidden message
    """
    # Make a copy of the image
    output = image.copy()
    
    # Convert message to binary
    binary_message = string_to_binary(message)
    
    # Add delimiter to know when message ends
    binary_message += '11111111'
    
    # Calculate maximum message size that can be embedded
    max_bytes = (image.shape[0] * image.shape[1] * 3 * bits_used) // 8
    if len(binary_message) > max_bytes * 8:
        raise ValueError(f"Message too long. Maximum {max_bytes} bytes allowed.")
    
    # Flatten the image
    output_flat = output.reshape(-1)
    
    # Define the bit mask based on number of bits used
    if bits_used == 1:
        mask = 0xFE  # 11111110 in binary
        divisor = 1
    else:  # bits_used == 2
        mask = 0xFC  # 11111100 in binary
        divisor = 2
    
    # Embed the message
    idx = 0
    for i in range(len(binary_message)):
        # Get the position in the flattened array
        pos = idx // divisor
        
        # If we've reached the end of the array, break
        if pos >= len(output_flat):
            break
        
        # Clear the last bits_used bits of the pixel value
        output_flat[pos] = output_flat[pos] & mask
        
        # Set the last bits_used bits to the message bits
        if bits_used == 1:
            output_flat[pos] = output_flat[pos] | int(binary_message[i])
        else:  # bits_used == 2
            if i % 2 == 0 and i+1 < len(binary_message):
                # Combine two bits from the message
                bits = int(binary_message[i:i+2], 2)
                output_flat[pos] = output_flat[pos] | bits
                i += 1  # Skip the next bit as we've already used it
        
        idx += bits_used
    
    # Reshape back to original image dimensions
    output = output_flat.reshape(image.shape)
    
    return output

def extract_message(image, bits_used=2):
    """
    Extract a hidden message from the least significant bits of the image
    
    Parameters:
    - image: Image with hidden message
    - bits_used: Number of LSBs used for hiding (1 or 2)
    
    Returns:
    - Extracted message
    """
    # Flatten the image
    flat_image = image.reshape(-1)
    
    # Create a bit mask based on bits_used
    if bits_used == 1:
        mask = 0x01  # 00000001 in binary
    else:  # bits_used == 2
        mask = 0x03  # 00000011 in binary
    
    # Extract the message bits
    binary = ""
    delimiter = "11111111"  # 8 ones to mark end of message
    
    i = 0
    while i < len(flat_image):
        # Extract bits
        if bits_used == 1:
            # Extract 1 bit
            bit = flat_image[i] & mask
            binary += str(bit)
        else:  # bits_used == 2
            # Extract 2 bits
            bits = flat_image[i] & mask
            binary += format(bits, f'0{bits_used}b')
        
        # Check for delimiter every 8 bits
        if len(binary) >= 8 and binary[-8:] == delimiter:
            binary = binary[:-8]  # Remove the delimiter
            break
        
        i += 1
    
    # Convert binary to string
    message = binary_to_string(binary)
    
    return message

def main():
    print("Steganography Application")
    print("------------------------")
    
    # Step 1: Load existing image
    print("\nStep 1: Loading existing selfie.jpg")
    image_path = 'selfie.jpg'
    
    if not os.path.exists(image_path):
        print(f"Error: {image_path} not found in the current directory.")
        return
    
    selfie = cv2.imread(image_path)
    if selfie is None:
        print(f"Error: Could not load {image_path}. Make sure it's a valid image file.")
        return
    
    original = selfie.copy()
    
    # Step 2: Get message from user
    print("\nStep 2: Enter message to hide")
    message = input("Enter the message to hide: ")
    
    # Step 3: Choose number of bits (1 or 2)
    bits_choice = input("Use 1 or 2 bits for hiding? (1/2) [default: 2]: ")
    bits_used = 1 if bits_choice == '1' else 2
    
    # Justify the choice
    print(f"\nUsing {bits_used} least significant bits for hiding the message.")
    print(f"Justification for using {bits_used} bits:")
    
    if bits_used == 1:
        print("- Better image quality (less visual distortion)")
        print("- Lower capacity (can hide less data)")
        print("- Higher security (harder to detect visually)")
    else:
        print("- Higher capacity (can hide more data)")
        print("- Slightly lower image quality")
        print("- Still provides good security with minimal visual distortion")
    
    max_bytes = (selfie.shape[0] * selfie.shape[1] * 3 * bits_used) // 8
    print(f"- Maximum message size with this image: {max_bytes} bytes")
    print(f"- Your message size: {len(message)} bytes")
    
    # Step 4: Hide the message
    print("\nStep 3: Hiding message in image...")
    try:
        output = hide_message(selfie, message, bits_used)
        print("Message hidden successfully!")
    except ValueError as e:
        print(f"Error: {e}")
        return
    
    # Step 5: Show both images
    print("\nStep 4: Displaying original and modified images...")
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.title("Image with Hidden Message")
    plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Step 6: Extract and display the hidden message
    print("\nStep 5: Extracting hidden message...")
    extracted = extract_message(output, bits_used)
    print(f"Extracted message: {extracted}")
    
    # Step 7: Save the modified image
    save_choice = input("\nWould you like to save the modified image? (y/n): ")
    if save_choice.lower() == 'y':
        cv2.imwrite("steganography_output.png", output)
        print("Modified image saved as 'steganography_output.png'")
    
    print("\nSteganography process completed.")

if __name__ == "__main__":
    main()

In [ ]:
from PIL import Image
import numpy as np

def embed_image(cover_image_path, secret_image_path, output_path):
    cover_image = Image.open(cover_image_path).convert("RGB")
    secret_image = Image.open(secret_image_path).convert("RGB")
    
    cover_pixels = np.array(cover_image)
    secret_pixels = np.array(secret_image)
    
    # Resize secret image to match cover image
    secret_pixels = np.resize(secret_pixels, (cover_pixels.shape[0], cover_pixels.shape[1], 3))
    
    height, width = secret_pixels.shape[:2]
    print(f"Secret Image Dimensions: Height={height}, Width={width}")
    
    # Encode dimensions in first row of the cover image
    cover_pixels[0, :4, 0] = [(height >> 8) & 255, height & 255, (width >> 8) & 255, width & 255]
    
    # Encode secret image using LSB
    for i in range(height):
        for j in range(width):
            for k in range(3):
                cover_pixels[i, j, k] = (cover_pixels[i, j, k] & 0xFE) | (secret_pixels[i, j, k] >> 7)
    
    stego_image = Image.fromarray(cover_pixels)
    stego_image.save(output_path)
    print(f"Stego image saved as {output_path}")

def extract_image(stego_image_path, output_path):
    stego_image = Image.open(stego_image_path).convert("RGB")
    stego_pixels = np.array(stego_image)
    
    # Extract dimensions from the first row
    height = (stego_pixels[0, 0, 0] << 8) | stego_pixels[0, 1, 0]
    width = (stego_pixels[0, 2, 0] << 8) | stego_pixels[0, 3, 0]
    
    print(f"Extracted Dimensions: Height={height}, Width={width}")
    
    if height == 0 or width == 0:
        raise ValueError("Invalid extracted dimensions. Data corruption may have occurred.")
    
    extracted_pixels = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Extract secret image from LSB
    for i in range(height):
        for j in range(width):
            for k in range(3):
                extracted_pixels[i, j, k] = (stego_pixels[i, j, k] & 1) * 255
    
    extracted_image = Image.fromarray(extracted_pixels)
    extracted_image.save(output_path)
    print(f"Extracted secret image saved as {output_path}")


embed_image("duck.jpg", "secretmessage.jpg", "stego_image.png")
extract_image("stego_image.png", "extracted_secret.png")


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

def main():
    while True:
        os.system('cls' if os.name == 'nt' else 'clear')
        print("\n===== STEGANOGRAPHY MENU =====")
        print("1. Superimpose Two Images")
        print("2. Exit")
        
        choice = input("\nEnter your choice (1-2): ")
        
        if choice == '1':
            superimpose_images()
        elif choice == '2':
            print("Thank you for using the Steganography Program!")
            break
        else:
            print("Invalid choice. Please try again.")
            input("Press Enter to continue...")

def superimpose_images():
    print("\n--- SUPERIMPOSE TWO IMAGES ---")
    
    # Get the first image
    img1_path = input("Enter the path to the first image (will be primarily visible): ")
    if not os.path.exists(img1_path):
        print(f"Error: The file '{img1_path}' does not exist.")
        input("Press Enter to continue...")
        return
    
    # Get the second image
    img2_path = input("Enter the path to the second image (will be hidden): ")
    if not os.path.exists(img2_path):
        print(f"Error: The file '{img2_path}' does not exist.")
        input("Press Enter to continue...")
        return
    
    try:
        # Load images using PIL first to handle different formats
        pil_img1 = Image.open(img1_path).convert('RGB')
        pil_img2 = Image.open(img2_path).convert('RGB')
        
        # Convert to numpy arrays for OpenCV processing
        img1 = np.array(pil_img1)
        img2 = np.array(pil_img2)
        
        # Resize second image to match dimensions of the first
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
        
        # Create output image
        result = np.zeros_like(img1)
        
        # Superimpose images using 4 most significant bits from img1 and 4 least significant bits from img2
        for i in range(3):  # For each color channel (RGB)
            # Keep the 4 most significant bits of the first image
            high_bits = (img1[:,:,i] & 0xF0)
            
            # Keep the 4 most significant bits of the second image, but shift them to the least significant positions
            low_bits = (img2[:,:,i] & 0xF0) >> 4
            
            # Combine the bits
            result[:,:,i] = high_bits | low_bits
        
        # Save the result
        output_path = "superimposed_image.png"
        cv2.imwrite(output_path, result)
        
        print(f"\nImages successfully superimposed and saved as '{output_path}'")
        
        # Display the original images and the result
        cv2.imshow("First Image (Primarily Visible)", img1)
        cv2.imshow("Second Image (Hidden)", img2)
        cv2.imshow("Superimposed Result", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
    input("\nPress Enter to continue...")

if __name__ == "__main__":
    main()